In [23]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip

In [24]:
# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
# data info
df_info = pd.read_csv('data_subc_sig.csv')
df_info.head()

,id,len,mean,std,id_person,id_location,id_direction,id_exp,id_paper,file,name,id_sign
0,S017_1_2_11,10258.0,22.066244,5.465441,17,1,2,11,IMG_0721,IMG_0721_10.png,강경휘,17
1,S017_1_2_11,10258.0,22.066244,5.465441,17,1,2,11,IMG_0721,IMG_0721_2.png,강경휘,17
2,S017_1_2_11,10258.0,22.066244,5.465441,17,1,2,11,IMG_0721,IMG_0721_3.png,강경휘,17
3,S017_1_2_11,10258.0,22.066244,5.465441,17,1,2,11,IMG_0721,IMG_0721_4.png,강경휘,17
4,S017_1_2_11,10258.0,22.066244,5.465441,17,1,2,11,IMG_0721,IMG_0721_5.png,강경휘,17


In [25]:
# read sample data
# load and uncompress.
with gzip.open(path_csi+df_info.iloc[0]['id']+'.pickle.gz','rb') as f:
    data1 = pickle.load(f)
data1_abs = np.abs(data1)
data1_ph = np.angle(data1)
data1_abs.shape

(10258, 30, 2, 3)

In [26]:
rms_ant = np.mean(0.707*np.abs(data1),axis=(0,1))
rms_ant

array([[15.13477315, 15.88365562, 16.19046086],
       [16.05749662, 15.44346479, 14.89515615]])

In [27]:
abs_sub = np.mean(np.abs(data1),axis=(0,2,3))
abs_sub

array([24.16307786, 23.54792958, 21.12386011, 22.8337564 , 25.34036246,
       26.56511735, 24.10576235, 21.30907425, 18.22157389, 15.70070364,
       16.42086644, 20.2196671 , 18.80152997, 14.17117523, 10.50370556,
       19.79207819, 19.26725541, 17.03863225, 20.53573555, 28.17584387,
       24.16751866, 26.01004285, 26.90871834, 27.32850079, 23.30399093,
       27.94756613, 26.19753246, 26.99139154, 24.67332286, 20.62102898])

In [28]:
data1_norm = data1/abs_sub[np.newaxis,:,np.newaxis,np.newaxis]
data1_norm.shape,np.mean(np.abs(data1_norm),axis=(0,2,3))

((10258, 30, 2, 3),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [29]:
data1[0,:,:,:]

array([[[ 10.67085966+15.7254774j ,  23.02659191 +8.98598709j,
          25.83471287 -7.8627387j ],
        [ 16.28710159-21.90334352j,   1.12324839-26.95796126j,
         -13.47898063-23.02659191j]],

       [[-21.90334352-13.47898063j, -25.27308868 +0.j        ,
         -21.90334352+11.79410805j],
        [-13.47898063+21.34171933j,  -1.68487258+23.5882161j ,
          10.10923547+20.78009513j]],

       [[ 18.53359836+14.04060482j,  22.46496771 +3.93136935j,
          23.02659191 -1.12324839j],
        [ 18.53359836-12.35573224j,  11.79410805-18.53359836j,
           2.24649677-21.90334352j]],

       [[ -8.98598709-20.21847094j, -17.41034998-14.04060482j,
         -23.5882161  -5.05461774j],
        [-23.02659191 +6.73949031j, -17.41034998+19.09522256j,
          -4.49299354+26.39633706j]],

       [[ 12.35573224+25.83471287j,  25.27308868+14.60222901j,
          29.20445803 -3.93136935j],
        [ 19.65684675-19.65684675j,   2.24649677-23.5882161j ,
          -5.05461774-19.0952

In [30]:
data1_grad = np.gradient(data1_norm,axis=0)
data1_grad.shape

(10258, 30, 2, 3)

In [119]:
# make data generator
def gen_csi(df_info,id_num):
    for file in set(df_info.id.values):
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
        data1_abs = np.abs(data1)
        data1_ph = np.angle(data1)
        data1_abs.shape

        # normalize through subc axis
        abs_sub = np.mean(np.abs(data1),axis=(0,2,3))
        data1_norm = data1/abs_sub[np.newaxis,:,np.newaxis,np.newaxis]

        # differentiation
        data1_grad = np.gradient(data1_norm,axis=0)

        # Label
        data1_lab = df_info[df_info.id==file]['id_person'].values[0]
        yield(data1_grad.reshape(-1,30,6),data1_lab)

In [122]:
#sample
#generator = gen_csi(df_info)
#a,b = next(generator)

In [123]:
a1 =a.reshape(-1,30,6) 

In [124]:
# parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 100
# Load Dataset
from sklearn.model_selection import train_test_split
tr_idx,te_idx = train_test_split(df_info.index,test_size=0.2,random_state=10)
id_num = len(np.unique(df_info.id_person))
df_train = df_info.loc[tr_idx]
df_test = df_info.loc[te_idx]

In [125]:
gen = lambda: (r for r in gen_csi(df_info))
train_dataset = tf.data.Dataset().from_generator(gen, (tf.float32,tf.int32)).repeat().batch(batch_size)

In [126]:
# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)
next_element = iterator.get_next()

In [127]:
# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)

In [130]:
def CNNmodel(in_data):
    input2d = tf.reshape(in_data, [-1,30,6])
    conv1 = tf.layers.conv1d(
        inputs=input2d, 
        filters=20, 
        kernel_size=5, 
        strides=1,
        #padding="same", 
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling1d(
        inputs=conv1, 
        pool_size=2, 
        strides=2)
        #padding="same")
    conv2 = tf.layers.conv1d(
        inputs=pool1, 
        filters=50, 
        kernel_size=5, 
        strides=1,
        #padding="same", 
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling1d(
        inputs=conv2, 
        pool_size=2, 
        strides=2)
        #padding="same")
    pool_flat = tf.layers.flatten(pool2)
    fc = tf.layers.dense(
        inputs= pool_flat, units=500, activation=tf.nn.relu)
    output = tf.layers.dense(inputs=fc, units=10)    
    return output

In [131]:
# define cost/loss & optimizer
logits = CNNmodel(next_element[0])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=tf.one_hot(next_element[1],id_num)))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [98]:
# accuracy
pred = tf.argmax(logits,1)
equal = tf.equal(pred,tf.argmax(next_element[1],1))
accuracy = tf.reduce_mean(tf.cast(equal,tf.float32))

#init
init_op = tf.global_variables_initializer()
training_init_op = iterator.make_initializer(train_dataset)

In [132]:
# run the training
epochs = training_epochs
results = pd.DataFrame(columns=['loss','accuracy'])
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i+1, l, acc * 100))
        '''
        # now setup the validation run
        test_iters = 1
        # re-initialize the iterator, but this time with validation data
        sess.run(test_init_op)
        test_l, test_acc = sess.run([loss, accuracy])
        print("Epoch: {}, test loss: {:.3f}, test accuracy: {:.2f}%".format(i+1, test_l, test_acc * 100))
        '''
        results.loc[i] = [l,acc]

FailedPreconditionError: Attempting to use uninitialized value beta1_power_1
	 [[{{node beta1_power_1/read}} = Identity[T=DT_FLOAT, _class=["loc:@Adam_1/Assign_1"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](beta1_power_1)]]

Caused by op 'beta1_power_1/read', defined at:
  File "C:\Users\herok\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\herok\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\herok\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\herok\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\herok\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\herok\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever